1. LOADING DEPENDICIES.

In [4]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [ ]:
summarizer = pipeline('summarization',model="facebook/bart-large-cnn")

2. SCRAPPING THE DATA FROM THE ONLINE BLOG POST 

In [4]:
url1 = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
url2 = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"
url3=  "https://writingcenter.unc.edu/tips-and-tools/literature-reviews/"

In [5]:
req = requests.get(url3)

In [ ]:
req.text

In [7]:
soup = BeautifulSoup(req.text,'html.parser')
results = soup.find_all(['h1','p'])

In [ ]:
results

In [ ]:
soup

In [10]:
text = [result.text for result in results]
article = ' '.join(text)

In [ ]:
text

In [ ]:
article

In [13]:
article = article.replace('.','.<eos>')
article = article.replace('!','!<eos>')
article = article.replace('?','?<eos>')

sentences = article.split('<eos>')

In [ ]:
sentences

3. CHUNKING THE DATA

In [ ]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
    if len(chunks) == current_chunk+1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

In [ ]:
chunks

In [17]:
for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [ ]:
chunk_id

In [ ]:
chunks[0]

In [ ]:
len(chunks[0].split(' '))

4. SUMMARIZING THE DATA USING TRANSFORMERS PIPELINE.

In [21]:
res = summarizer(chunks,max_length=120,min_length=30,do_sample=False)

In [ ]:
res

In [ ]:
' '.join([summ['summary_text'] for summ in res])

5. CONVERT THE SUMMARY INTO .TXT FILE TO READ.

In [24]:
texts = ' '.join([summ['summary_text'] for summ in res])


In [25]:
with open('literaturesummary.txt','w') as f:
    f.write(texts)